<a href="https://colab.research.google.com/github/bajrangimishra9/Arduino-Tinkercad-_Projects/blob/main/Amazon_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'amazon-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5699506%2F9391920%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240916%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240916T065303Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6836ce45a757473268770024a8452bc853aa8e34da5043624cfd8164d42cb2ccb3a9817f2f4a1122bf2bf00b666859400f01ade8f6fe701ff3f04cebe3962021f310b94b2606f8986a0c46dc092e4ca64fe6c95ca4a430ecfb0271bb3a4e962ce217cc3ae92e495d216a7d80a88d01df034a365ec853c07fca7d3da69e74954091a54d945e9a0cc6784b9f61fc5e4ec085ee098b405a84d00824bc5149a6390d3f733251190b895b7a90cc651c5f3e48c977484209896d045047be154d3882a68dcdb85e6328dd928295452e4e2050189cab66a6bf396336b9f451c49092724939dcae3f0ada2110af2947bf79af8ca5058e25662c4e89c1c644d91d235a3d65,test-dataset-file:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5711691%2F9407156%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240916%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240916T065303Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D31d3bd4d38d0c48d19dfbfd8ad1315a79fb96bda8d4f5129b9abfd1e8ed15901b8dc20aeab6d779fd37743ce31edb03eaa36d644a8c164b8dd235f9afae641f927294834902f00ccd66aa0095456af4a647f0b5690d6acd1e0013c40fa7acc3dba0ba0e9109802e185c45b160d4e2408a05cdcf722c2d70a24fb3e9b90322280b15a227af0ab859be1d5f26184ae90713b1a403f97bb7d86ea66624b004803d403359f607473bc0aa45632fbfa3399c13816a16cdc998c80090668d01ee1906fe98cf790a2938a8fa45fdd5eb226a85b24d52c980fb1a62b812fa1d070724d7124b96bb9decf1c424600b05919f0a10c929ff366b813c6178a027a873c783a57,after-training:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5713039%2F9408839%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240916%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240916T065303Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D63aab3eb652e06c0428686b6ccf8e4cee6f5a68c1e1265d2fcc7e31302172b8b2b363193ee34ab3edee5f498e211835aa093c6d74d45e66a6fae643fe916b48f9f805f69cdf78b79dce258af6c08fac0cb242331af26d24caa6fbf82fc1c5206c579c4595ee398a6617b0a90db7c668e7a8e90b80064109e830758c5f3ea0d9e55a6c1131038afd46b384d631c7570bec12083b531a1f557dc236af66899c19d27d467b58097efead684ea48170ee2d0ef2b7ae2e4c4dcb2435b288b29da165e9e45b92670afca376e775f649b1d083749e994d721a6e9d711fbd3a90ac637d2e0cb4178717c1340272e456fa161d701b3788f3076e6d03615397eec5de0f254'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-dataset-file/dataset_part2.csv
/kaggle/input/after-training/trained_model.pkl
/kaggle/input/after-training/vectorizer.pkl
/kaggle/input/after-training/predicted_units.csv
/kaggle/input/amazon-dataset/dataset/._sample_test_out.csv
/kaggle/input/amazon-dataset/dataset/._train.csv
/kaggle/input/amazon-dataset/dataset/._sample_test.csv
/kaggle/input/amazon-dataset/dataset/._sample_test_out_fail.csv
/kaggle/input/amazon-dataset/dataset/._test.csv
/kaggle/input/amazon-dataset/src/._constants.py
/kaggle/input/amazon-dataset/src/._utils.py
/kaggle/input/amazon-dataset/src/._sanity.py
/kaggle/input/amazon-dataset/src/._test.ipynb
/kaggle/input/amazon-dataset/student_resource 3/._src
/kaggle/input/amazon-dataset/student_resource 3/._sample_code.py
/kaggle/input/amazon-dataset/student_resource 3/README.md
/kaggle/input/amazon-dataset/student_resource 3/._README.md
/kaggle/input/amazon-dataset/student_resource 3/._dataset
/kaggle/input/amazon-dataset/student_resource 3/sample_co

In [ ]:
df = pd.read_csv('/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv')

In [ ]:
df.head()

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram


In [ ]:
pip install easyocr

Note: you may need to restart the kernel to use updated packages.


# For single image extraction

1) To see detailed extraction

In [ ]:
"""import easyocr
import pandas as pd
from PIL import Image
import numpy as np
import requests
from io import BytesIO
import re

# Initialize the EasyOCR reader (CPU mode if GPU is not available or has low memory)
reader = easyocr.Reader(['en'], gpu=True)

# Define regex patterns based on the allowed units
patterns = {
    'gram': r'(\d+(\.\d+)?)\s*(g|grams?)',
    'kilogram': r'(\d+(\.\d+)?)\s*(kg|kilograms?)',
    'milligram': r'(\d+(\.\d+)?)\s*(mg|milligrams?)',
    'microgram': r'(\d+(\.\d+)?)\s*(µg|micrograms?)',
    'ounce': r'(\d+(\.\d+)?)\s*(oz|ounces?)',
    'pound': r'(\d+(\.\d+)?)\s*(lb|pounds?)',
    'ton': r'(\d+(\.\d+)?)\s*(ton|tons?)',
    'centimetre': r'(\d+(\.\d+)?)\s*(cm|centimetre|centimeters?)',
    'foot': r'(\d+(\.\d+)?)\s*(ft|foot|feet)',
    'inch': r'(\d+(\.\d+)?)\s*(in|inch|inches)',
    'metre': r'(\d+(\.\d+)?)\s*(m|metre|meters?)',
    'millimetre': r'(\d+(\.\d+)?)\s*(mm|millimetre|millimeters?)',
    'yard': r'(\d+(\.\d+)?)\s*(yd|yards?)',
    'kilovolt': r'(\d+(\.\d+)?)\s*(kv|kilovolts?)',
    'millivolt': r'(\d+(\.\d+)?)\s*(mv|millivolts?)',
    'volt': r'(\d+(\.\d+)?)\s*(v|volts?)',
    'kilowatt': r'(\d+(\.\d+)?)\s*(kw|kilowatts?)',
    'watt': r'(\d+(\.\d+)?)\s*(w|watts?)',
    'centilitre': r'(\d+(\.\d+)?)\s*(cl|centilitres?)',
    'cubic foot': r'(\d+(\.\d+)?)\s*(cubic foot|cubic feet)',
    'cubic inch': r'(\d+(\.\d+)?)\s*(cubic inch|cubic inches)',
    'cup': r'(\d+(\.\d+)?)\s*(cup|cups?)',
    'decilitre': r'(\d+(\.\d+)?)\s*(dl|decilitres?)',
    'fluid ounce': r'(\d+(\.\d+)?)\s*(fl oz|fluid ounces?)',
    'gallon': r'(\d+(\.\d+)?)\s*(gallon|gallons?)',
    'imperial gallon': r'(\d+(\.\d+)?)\s*(imperial gallon|imperial gallons?)',
    'litre': r'(\d+(\.\d+)?)\s*(l|litre|liters?)',
    'microlitre': r'(\d+(\.\d+)?)\s*(µl|microlitre|microlitres?)',
    'millilitre': r'(\d+(\.\d+)?)\s*(ml|millilitre|millilitres?)',
    'pint': r'(\d+(\.\d+)?)\s*(pt|pints?)',
    'quart': r'(\d+(\.\d+)?)\s*(qt|quarts?)',
}

# Map entity names to their corresponding unit patterns
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Allowed units set based on entity_unit_map
allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

# Function to download an image from a URL
def download_image(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img

# Preprocessing function to clean up OCR text
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()

    # Replace "O" with "0" (to handle common OCR errors like "140OmG")
    text = text.replace("o", "0")

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Remove any unnecessary punctuation that might interfere with matching
    text = re.sub(r'[^\w\s.,]', '', text)

    return text

# Function to extract various units from the text
def extract_units(text, entity_name):
    extracted_units = set()

    # Preprocess the text before extracting units
    cleaned_text = preprocess_text(text)

    if entity_name in entity_unit_map:
        unit_list = entity_unit_map[entity_name]
        for unit in unit_list:
            if unit in patterns:
                pattern = patterns[unit]
                matches = re.findall(pattern, cleaned_text)
                if matches:
                    # Format as "value unit" and store unique entries
                    for match in matches:
                        value = float(match[0])
                        formatted_entry = f"{value} {unit}"
                        extracted_units.add(formatted_entry)

    return extracted_units

# Function to rotate the image and check OCR text
def rotate_and_extract(image):
    # Initialize variables to track the best result
    best_text = ""
    best_angle = 0

    # Try OCR on the original and rotated images
    for angle in [0, 90, 180, 270]:
        rotated_img = image.rotate(angle, expand=True)

        # Convert the rotated image to a numpy array for easyocr
        rotated_img_np = np.array(rotated_img)

        # Extract text from rotated image
        img_text = reader.readtext(rotated_img_np)
        final_text = " ".join([text for _, text, __ in img_text])

        if len(final_text.strip()) > len(best_text.strip()):  # Choose the best text based on length
            best_text = final_text
            best_angle = angle

    # Return the best text and angle
    return best_text, best_angle

# Function to process each image URL
def process_image_url(image_url, entity_name):
    # Download and process the image
    image = download_image(image_url)
    final_text, best_angle = rotate_and_extract(image)

    # Extract units from the final text based on the entity_name
    units = extract_units(final_text, entity_name)

    return final_text, best_angle, units

# Load the CSV file
csv_file_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv'
df = pd.read_csv(csv_file_path)

# Limit to first 30 links for processing
df = df.head(30)

# Initialize a list to store results
results = []

# Iterate through each image URL in the CSV file
for index, row in df.iterrows():
    image_url = row['image_link']  # Assuming the CSV column is named 'image_link'
    entity_name = row['entity_name']  # Assuming this column specifies what entity to extract (e.g., weight, volume)

    print(f"Processing image from URL: {image_url} for entity: {entity_name}")

    final_text, best_angle, units = process_image_url(image_url, entity_name)

    # Prepare the result row
    result_row = {
        'image_link': image_url,
        'entity_name': entity_name,
        'extracted_text': final_text,
        'best_angle': best_angle,
        'predicted': list(units)  # Convert set to list for the predicted result
    }

    # Append the result to the list
    results.append(result_row)

    # Print results for each image
    print(f"Extracted Text (Best angle {best_angle} degrees): {final_text}")
    print(f"Extracted Units for {entity_name}: {result_row['predicted']}")
    print("--------------------------------------------------")

# Create a new DataFrame from results and save it as a CSV
results_df = pd.DataFrame(results)
results_df.to_csv('predicted_units.csv', index=False)
"""

'import easyocr\nimport pandas as pd\nfrom PIL import Image\nimport numpy as np\nimport requests\nfrom io import BytesIO\nimport re\n\n# Initialize the EasyOCR reader (CPU mode if GPU is not available or has low memory)\nreader = easyocr.Reader([\'en\'], gpu=True)\n\n# Define regex patterns based on the allowed units\npatterns = {\n    \'gram\': r\'(\\d+(\\.\\d+)?)\\s*(g|grams?)\',\n    \'kilogram\': r\'(\\d+(\\.\\d+)?)\\s*(kg|kilograms?)\',\n    \'milligram\': r\'(\\d+(\\.\\d+)?)\\s*(mg|milligrams?)\',\n    \'microgram\': r\'(\\d+(\\.\\d+)?)\\s*(µg|micrograms?)\',\n    \'ounce\': r\'(\\d+(\\.\\d+)?)\\s*(oz|ounces?)\',\n    \'pound\': r\'(\\d+(\\.\\d+)?)\\s*(lb|pounds?)\',\n    \'ton\': r\'(\\d+(\\.\\d+)?)\\s*(ton|tons?)\',\n    \'centimetre\': r\'(\\d+(\\.\\d+)?)\\s*(cm|centimetre|centimeters?)\',\n    \'foot\': r\'(\\d+(\\.\\d+)?)\\s*(ft|foot|feet)\',\n    \'inch\': r\'(\\d+(\\.\\d+)?)\\s*(in|inch|inches)\',\n    \'metre\': r\'(\\d+(\\.\\d+)?)\\s*(m|metre|meters?)\',\n    \'millimetr

2)Didn't want to see details

In [ ]:
import easyocr
import pandas as pd
from PIL import Image
import numpy as np
import urllib
import re
import warnings
import os
import multiprocessing
from functools import partial
from tqdm import tqdm
from pathlib import Path
import time

# Suppress warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=True)

# Regex patterns for units
patterns = {
    'gram': r'(\d+(\.\d+)?)\s*(g|grams?)',
    'kilogram': r'(\d+(\.\d+)?)\s*(kg|kilograms?)',
    'milligram': r'(\d+(\.\d+)?)\s*(mg|milligrams?)',
    'microgram': r'(\d+(\.\d+)?)\s*(µg|micrograms?)',
    'ounce': r'(\d+(\.\d+)?)\s*(oz|ounces?)',
    'pound': r'(\d+(\.\d+)?)\s*(lb|pounds?)',
    'ton': r'(\d+(\.\d+)?)\s*(ton|tons?)',
    'centimetre': r'(\d+(\.\d+)?)\s*(cm|centimetre|centimeters?)',
    'foot': r'(\d+(\.\d+)?)\s*(ft|foot|feet)',
    'inch': r'(\d+(\.\d+)?)\s*(in|inch|inches)',
    'metre': r'(\d+(\.\d+)?)\s*(m|metre|meters?)',
    'millimetre': r'(\d+(\.\d+)?)\s*(mm|millimetre|millimeters?)',
    'yard': r'(\d+(\.\d+)?)\s*(yd|yards?)',
    'kilovolt': r'(\d+(\.\d+)?)\s*(kv|kilovolts?)',
    'millivolt': r'(\d+(\.\d+)?)\s*(mv|millivolts?)',
    'volt': r'(\d+(\.\d+)?)\s*(v|volts?)',
    'kilowatt': r'(\d+(\.\d+)?)\s*(kw|kilowatts?)',
    'watt': r'(\d+(\.\d+)?)\s*(w|watts?)',
    'centilitre': r'(\d+(\.\d+)?)\s*(cl|centilitres?)',
    'cubic foot': r'(\d+(\.\d+)?)\s*(cubic foot|cubic feet)',
    'cubic inch': r'(\d+(\.\d+)?)\s*(cubic inch|cubic inches)',
    'cup': r'(\d+(\.\d+)?)\s*(cup|cups?)',
    'decilitre': r'(\d+(\.\d+)?)\s*(dl|decilitres?)',
    'fluid ounce': r'(\d+(\.\d+)?)\s*(fl oz|fluid ounces?)',
    'gallon': r'(\d+(\.\d+)?)\s*(gallon|gallons?)',
    'imperial gallon': r'(\d+(\.\d+)?)\s*(imperial gallon|imperial gallons?)',
    'litre': r'(\d+(\.\d+)?)\s*(l|litre|liters?)',
    'microlitre': r'(\d+(\.\d+)?)\s*(µl|microlitre|microlitres?)',
    'millilitre': r'(\d+(\.\d+)?)\s*(ml|millilitre|millilitres?)',
    'pint': r'(\d+(\.\d+)?)\s*(pt|pints?)',
    'quart': r'(\d+(\.\d+)?)\s*(qt|quarts?)',
}

# Map entity names to their corresponding unit patterns
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

# Allowed units
allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

# Download an image from URL with retries and placeholder on failure
def download_image(image_link, save_folder, retries=3, delay=3):
    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    # Skip if image already exists
    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except Exception:
            time.sleep(delay)

    # Create a placeholder image in case of failure
    Image.new('RGB', (100, 100), color='black').save(image_save_path)

# Parallel image downloading function
def download_images(image_links, download_folder):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    download_image_partial = partial(download_image, save_folder=download_folder)
    with multiprocessing.Pool(64) as pool:
        list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
        pool.close()
        pool.join()

# Preprocess text (lowercase, handle common OCR errors)
def preprocess_text(text):
    text = text.lower().replace("o", "0")
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s.,]', '', text)
    return text

# Extract units from text based on entity type
def extract_units(text, entity_name):
    extracted_units = set()
    cleaned_text = preprocess_text(text)

    if entity_name in entity_unit_map:
        for unit in entity_unit_map[entity_name]:
            if unit in patterns:
                matches = re.findall(patterns[unit], cleaned_text)
                for match in matches:
                    value = float(match[0])
                    extracted_units.add(f"{value} {unit}")
    return extracted_units

# Perform OCR and extract best text by rotating image
def rotate_and_extract(image):
    best_text, best_angle = "", 0
    for angle in [0, 90, 180, 270]:
        rotated_img = image.rotate(angle, expand=True)
        rotated_img_np = np.array(rotated_img)
        img_text = reader.readtext(rotated_img_np)
        final_text = " ".join([text for _, text, __ in img_text])
        if len(final_text.strip()) > len(best_text.strip()):
            best_text, best_angle = final_text, angle
    return best_text, best_angle

# Process image URL
def process_image_url(image_url, entity_name, download_folder):
    # Download and open image
    filename = Path(image_url).name
    image_path = os.path.join(download_folder, filename)
    image = Image.open(image_path)

    # Extract text using OCR and find best rotation
    final_text, best_angle = rotate_and_extract(image)

    # Extract units
    units = extract_units(final_text, entity_name)
    return final_text, best_angle, units

# Load CSV file (replace with your path)
csv_file_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/train.csv'
df = pd.read_csv(csv_file_path).head(500)  # Process first 100 links

# Image folder to save the downloaded images
download_folder = "downloaded_images"

# Download images in parallel
print("Starting image download...")
download_images(df['image_link'], download_folder)

# Initialize list to store results
results = []

# Start tracking time
start_time = time.time()

# Process each image and extract relevant data
for index, row in df.iterrows():
    image_url = row['image_link']
    entity_name = row['entity_name']

    # Process image to get text, angle, and units
    final_text, best_angle, units = process_image_url(image_url, entity_name, download_folder)

    # Add results
    result_row = {
        'image_link': image_url,
        'entity_name': entity_name,
        'extracted_text': final_text,
        'best_angle': best_angle,
        'predicted': list(units)
    }
    results.append(result_row)

    # Calculate elapsed time
    elapsed_time = time.time() - start_time
    remaining_images = len(df) - (index + 1)
    estimated_total_time = (elapsed_time / (index + 1)) * len(df)
    remaining_time = estimated_total_time - elapsed_time

    # Print countdown
    print(f"Processed {index + 1}/{len(df)} images. Estimated time remaining: {remaining_time // 60:.0f}m {remaining_time % 60:.0f}s")

# Create DataFrame and save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('predicted_units.csv', index=False)

print(f"Processed {len(results)} images.")

Starting image download...


100%|██████████| 500/500 [00:04<00:00, 119.91it/s]


Processed 1/500 images. Estimated time remaining: 56m 23s
Processed 2/500 images. Estimated time remaining: 37m 31s
Processed 3/500 images. Estimated time remaining: 38m 43s
Processed 4/500 images. Estimated time remaining: 41m 14s
Processed 5/500 images. Estimated time remaining: 37m 14s
Processed 6/500 images. Estimated time remaining: 35m 46s
Processed 7/500 images. Estimated time remaining: 47m 6s
Processed 8/500 images. Estimated time remaining: 44m 39s
Processed 9/500 images. Estimated time remaining: 46m 46s
Processed 10/500 images. Estimated time remaining: 45m 6s
Processed 11/500 images. Estimated time remaining: 43m 28s
Processed 12/500 images. Estimated time remaining: 41m 0s
Processed 13/500 images. Estimated time remaining: 44m 28s
Processed 14/500 images. Estimated time remaining: 45m 49s
Processed 15/500 images. Estimated time remaining: 63m 48s
Processed 16/500 images. Estimated time remaining: 65m 24s
Processed 17/500 images. Estimated time remaining: 63m 25s
Processed

In [ ]:
# Import necessary libraries for model training
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
import pandas as pd

# Load the processed training data
train_data = pd.read_csv('predicted_units.csv')

# Prepare features (extracted_text) and labels (entity_name)
X_train = train_data['extracted_text'].astype(str)  # Ensure all values are strings
y_train = train_data['entity_name']

# Convert the text data to numerical format using TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)

# Split the training data into training and validation sets
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_tfidf, y_train, test_size=0.2, random_state=42)

# Train a RandomForest Classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_split, y_train_split)

# Validate the model using validation data
y_pred = clf.predict(X_val_split)
accuracy = accuracy_score(y_val_split, y_pred)
print(f"Validation Accuracy: {accuracy:.2f}")

# Save the trained model and vectorizer for later use
joblib.dump(clf, 'trained_model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')


Validation Accuracy: 0.82


['vectorizer.pkl']

In [ ]:
import pandas as pd
import urllib.request
from PIL import Image
import easyocr
import re
import concurrent.futures
import time
import io

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Load test CSV file
test_csv_file_path = '/kaggle/input/amazon-dataset/student_resource 3/dataset/sample_test.csv'
test_df = pd.read_csv(test_csv_file_path)

# Function to clean misinterpreted values (e.g., "2OCM 1SCM" -> "20cm 1cm")
def correct_misinterpretations(text):
    text = text.upper()
    text = text.replace("O", "0")  # Replace O with 0
    text = text.replace("SCM", "CM")  # Correct SCM to CM
    return text

# Updated function to extract units and their values from the text (including misinterpretation handling)
def extract_units(text):
    text = correct_misinterpretations(text)
    patterns = {
        'gram': r'(\d+(\.\d+)?)\s*(g(rams?)?)',
        'kilogram': r'(\d+(\.\d+)?)\s*(kg)',
        'milligram': r'(\d+(\.\d+)?)\s*(mg)',
        'microgram': r'(\d+(\.\d+)?)\s*(µg)',
        'ounce': r'(\d+(\.\d+)?)\s*(oz)',
        'pound': r'(\d+(\.\d+)?)\s*(lb|pounds?)',
        'ton': r'(\d+(\.\d+)?)\s*(ton)',
        'centimetre': r'(\d+(\.\d+)?)\s*(cm)',
        'foot': r'(\d+(\.\d+)?)\s*(ft|feet|foot)',
        'inch': r'(\d+(\.\d+)?)\s*(in(ch)?|")',
        'metre': r'(\d+(\.\d+)?)\s*(m(etres?)?)',
        'millimetre': r'(\d+(\.\d+)?)\s*(mm)',
        'yard': r'(\d+(\.\d+)?)\s*(yd)',
        'kilovolt': r'(\d+(\.\d+)?)\s*(kv)',
        'millivolt': r'(\d+(\.\d+)?)\s*(mv)',
        'volt': r'(\d+(\.\d+)?)\s*(v)',
        'kilowatt': r'(\d+(\.\d+)?)\s*(kw)',
        'watt': r'(\d+(\.\d+)?)\s*(w)',
        'litre': r'(\d+(\.\d+)?)\s*(l(itres?)?)',
        'millilitre': r'(\d+(\.\d+)?)\s*(ml)'
    }
    units = set()
    text = text.lower()
    for unit_name, pattern in patterns.items():
        matches = re.findall(pattern, text, re.IGNORECASE)
        for match in matches:
            extracted_value = f"{match[0]} {unit_name}" if unit_name != 'inch' else f"{match[0]} inch"
            units.add(extracted_value)
    return units

# Function to process images directly from URL without saving them
def process_image_from_url(image_link):
    try:
        with urllib.request.urlopen(image_link) as url:
            image = Image.open(io.BytesIO(url.read()))
        result = reader.readtext(image, detail=0)
        final_text = " ".join(result)
        return final_text
    except Exception as e:
        print(f"Error processing image {image_link}: {e}")
        return ""

# Function to match the extracted units with the entity name
def match_units_with_entity(entity_name, extracted_units):
    entity_unit_map = {
        'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
        'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
        'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
        'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
        'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
        'voltage': {'kilovolt', 'millivolt', 'volt'},
        'wattage': {'kilowatt', 'watt'},
        'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
    }
    valid_units = entity_unit_map.get(entity_name, set())
    relevant_units = [unit for unit in extracted_units if any(valid_unit in unit for valid_unit in valid_units)]
    return relevant_units

# Function to process each row and return results
def process_row(row):
    image_url = row['image_link']
    entity_name = row['entity_name']
    group_id = row['group_id']
    final_text = process_image_from_url(image_url)
    extracted_units = extract_units(final_text)
    matched_units = match_units_with_entity(entity_name, extracted_units)
    predicted_value = ", ".join(matched_units) if matched_units else ""
    return {
        'image_link': image_url,
        'entity_name': entity_name,
        'group_id': group_id,
        'extracted_text': final_text,
        'predicted_units': matched_units,
        'predicted': predicted_value
    }

# Initialize list to store test results
test_results = []

# Start tracking time
start_time = time.time()

# Use ThreadPoolExecutor for concurrent processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Process each row in parallel
    future_to_row = {executor.submit(process_row, row): row for index, row in test_df.iterrows()}

    for index, future in enumerate(concurrent.futures.as_completed(future_to_row)):
        try:
            result_row = future.result()
            test_results.append(result_row)

            # Calculate elapsed time and remaining time
            elapsed_time = time.time() - start_time
            remaining_images = len(test_df) - (index + 1)
            estimated_total_time = (elapsed_time / (index + 1)) * len(test_df)
            remaining_time = estimated_total_time - elapsed_time

            # Print progress
            print(f"Processed {index + 1}/{len(test_df)} test images. Estimated time remaining: {remaining_time // 60:.0f}m {remaining_time % 60:.0f}s")
        except Exception as e:
            print(f"Error processing row: {e}")

# Calculate final elapsed time
total_elapsed_time = time.time() - start_time

# Create DataFrame and save test results to CSV
test_results_df = pd.DataFrame(test_results)
test_results_df.to_csv('test_matched_units.csv', index=False)

print(f"Processed {len(test_results)} test images in {total_elapsed_time // 60:.0f}m {total_elapsed_time % 60:.0f}s")


/opt/conda/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

Processed 1/88 test images. Estimated time remaining: 4m 13s
Processed 2/88 test images. Estimated time remaining: 2m 5s
Processed 3/88 test images. Estimated time remaining: 1m 23s
Processed 4/88 test images. Estimated time remaining: 1m 16s
Processed 5/88 test images. Estimated time remaining: 1m 4s
Processed 6/88 test images. Estimated time remaining: 0m 54s
Processed 7/88 test images. Estimated time remaining: 0m 45s
Processed 8/88 test images. Estimated time remaining: 0m 39s
Processed 9/88 test images. Estimated time remaining: 0m 48s
Processed 10/88 test images. Estimated time remaining: 0m 43s
Processed 11/88 test images. Estimated time remaining: 0m 39s
Processed 12/88 test images. Estimated time remaining: 0m 35s


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:917: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1424.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


Processed 13/88 test images. Estimated time remaining: 0m 37s
Processed 14/88 test images. Estimated time remaining: 0m 37s
Processed 15/88 test images. Estimated time remaining: 0m 36s
Processed 16/88 test images. Estimated time remaining: 0m 33s
Processed 17/88 test images. Estimated time remaining: 0m 31s
Processed 18/88 test images. Estimated time remaining: 0m 29s
Processed 19/88 test images. Estimated time remaining: 0m 28s
Processed 20/88 test images. Estimated time remaining: 0m 27s
Processed 21/88 test images. Estimated time remaining: 0m 26s
Processed 22/88 test images. Estimated time remaining: 0m 25s
Processed 23/88 test images. Estimated time remaining: 0m 25s
Processed 24/88 test images. Estimated time remaining: 0m 24s
Processed 25/88 test images. Estimated time remaining: 0m 23s
Processed 26/88 test images. Estimated time remaining: 0m 22s
Processed 27/88 test images. Estimated time remaining: 0m 21s
Processed 28/88 test images. Estimated time remaining: 0m 21s
Processe

In [ ]:
final_df = pd.read_csv('test_matched_units.csv')

In [ ]:
final_df.head(30)

,image_link,entity_name,group_id,extracted_text,predicted_units,predicted
0,https://m.media-amazon.com/images/I/41-NCxNuBx...,width,658003,2OCM 1SCM,"['1 centimetre', '20 centimetre']","1 centimetre, 20 centimetre"
1,https://m.media-amazon.com/images/I/41-NCxNuBx...,depth,658003,2OCM 1SCM,"['1 centimetre', '20 centimetre']","1 centimetre, 20 centimetre"
2,https://m.media-amazon.com/images/I/41ADVPQgZO...,item_weight,993359,CalaLrian Chili Powder,[],NaN
3,https://m.media-amazon.com/images/I/417NJrPEk+...,maximum_weight_recommendation,939426,Deodorizing module Cat litter shovel Adsorb ba...,[],NaN
4,https://m.media-amazon.com/images/I/41nblnEkJ3...,voltage,648011,LED OSRAM 55w= 40w 470 Im am White B22d,[],NaN
5,https://m.media-amazon.com/images/I/41pvwR9Gba...,voltage,965518,LED WORK LIGHT for Dark Areas 'DAKLAR |,[],NaN
6,https://m.media-amazon.com/images/I/41nblnEkJ3...,wattage,648011,LED OSRAM 55w= 40w 470 Im am White B22d,"['55 watt', '40 watt']","55 watt, 40 watt"
7,https://m.media-amazon.com/images/I/41uwo4PVnu...,depth,640565,"Joz"" [",[],NaN
8,https://m.media-amazon.com/images/I/41ygXRvf8l...,height,752266,50cm 8cm,"['50 centimetre', '8 centimetre']","50 centimetre, 8 centimetre"
9,https://m.media-amazon.com/images/I/41uwo4PVnu...,width,640565,"Joz"" [",[],NaN


In [ ]:
# Convert 'predicted' column from a list of strings to a single string without brackets and quotes
final_df['predicted_units'] = final_df['predicted_units'].apply(lambda x: ', '.join(sorted(set(x.strip('[]').replace("'", "").split(', ')))))

In [ ]:
df = final_df

In [ ]:
df.head(10)

,image_link,entity_name,group_id,extracted_text,predicted_units,predicted
0,https://m.media-amazon.com/images/I/41-NCxNuBx...,width,658003,2OCM 1SCM,"1 centimetre, 20 centimetre","1 centimetre, 20 centimetre"
1,https://m.media-amazon.com/images/I/41-NCxNuBx...,depth,658003,2OCM 1SCM,"1 centimetre, 20 centimetre","1 centimetre, 20 centimetre"
2,https://m.media-amazon.com/images/I/41ADVPQgZO...,item_weight,993359,CalaLrian Chili Powder,,NaN
3,https://m.media-amazon.com/images/I/417NJrPEk+...,maximum_weight_recommendation,939426,Deodorizing module Cat litter shovel Adsorb ba...,,NaN
4,https://m.media-amazon.com/images/I/41nblnEkJ3...,voltage,648011,LED OSRAM 55w= 40w 470 Im am White B22d,,NaN
5,https://m.media-amazon.com/images/I/41pvwR9Gba...,voltage,965518,LED WORK LIGHT for Dark Areas 'DAKLAR |,,NaN
6,https://m.media-amazon.com/images/I/41nblnEkJ3...,wattage,648011,LED OSRAM 55w= 40w 470 Im am White B22d,"40 watt, 55 watt","55 watt, 40 watt"
7,https://m.media-amazon.com/images/I/41uwo4PVnu...,depth,640565,"Joz"" [",,NaN
8,https://m.media-amazon.com/images/I/41ygXRvf8l...,height,752266,50cm 8cm,"50 centimetre, 8 centimetre","50 centimetre, 8 centimetre"
9,https://m.media-amazon.com/images/I/41uwo4PVnu...,width,640565,"Joz"" [",,NaN


In [ ]:
import pandas as pd
import re

def extract_number(value):
    """Extract the number from a value string."""
    match = re.search(r'(\d+\.?\d*)', value)
    if match:
        return float(match.group(1))
    return None

def convert_to_base_unit(value, unit):
    """Convert the value to a base unit (e.g., centimeters)."""
    conversions = {
        'cm': 1,
        'centimetre': 1,
        'inch': 2.54,
        'foot': 30.48,
        'yard': 91.44,
        'watt': 1, # Assuming 'watt' does not need conversion
        'gram': 1,
        'kilogram': 1000,
        'ounce': 28.3495,
        'pound': 453.592
    }
    return value * conversions.get(unit, 1)

def find_max_value(predicted_units):
    """Find the maximum value in the predicted_units column."""
    max_value = -float('inf')
    max_unit = None

    if not predicted_units:
        return None

    parts = predicted_units.split(',')
    for part in parts:
        number_match = re.search(r'(\d+\.?\d*)', part)
        unit_match = re.search(r'(cm|centimetre|inch|foot|yard|watt|gram|kilogram|ounce|pound|meter|metre|volt)', part, re.IGNORECASE)
        if number_match and unit_match:
            number = float(number_match.group(1))
            unit = unit_match.group(1).lower()
            value_in_base = convert_to_base_unit(number, unit)
            if value_in_base > max_value:
                max_value = value_in_base
                max_unit = part
    return max_unit if max_unit else None

# Apply the function
df['max_predicted_units'] = df['predicted_units'].apply(lambda x: find_max_value(x))

print(df[['image_link', 'entity_name', 'predicted_units', 'max_predicted_units']].head(30))


                                           image_link  \
0   https://m.media-amazon.com/images/I/41-NCxNuBx...   
1   https://m.media-amazon.com/images/I/41-NCxNuBx...   
2   https://m.media-amazon.com/images/I/41ADVPQgZO...   
3   https://m.media-amazon.com/images/I/417NJrPEk+...   
4   https://m.media-amazon.com/images/I/41nblnEkJ3...   
5   https://m.media-amazon.com/images/I/41pvwR9Gba...   
6   https://m.media-amazon.com/images/I/41nblnEkJ3...   
7   https://m.media-amazon.com/images/I/41uwo4PVnu...   
8   https://m.media-amazon.com/images/I/41ygXRvf8l...   
9   https://m.media-amazon.com/images/I/41uwo4PVnu...   
10  https://m.media-amazon.com/images/I/41ygXRvf8l...   
11  https://m.media-amazon.com/images/I/41zgjN+zW3...   
12  https://m.media-amazon.com/images/I/41o3iis9E7...   
13  https://m.media-amazon.com/images/I/51+oHGvSvu...   
14  https://m.media-amazon.com/images/I/417SThj+Sr...   
15  https://m.media-amazon.com/images/I/510xYFNYQ8...   
16  https://m.media-amazon.com/

In [ ]:
df.head(30)

,image_link,entity_name,group_id,extracted_text,predicted_units,predicted,max_predicted_units
0,https://m.media-amazon.com/images/I/41-NCxNuBx...,width,658003,2OCM 1SCM,"1 centimetre, 20 centimetre","1 centimetre, 20 centimetre",20 centimetre
1,https://m.media-amazon.com/images/I/41-NCxNuBx...,depth,658003,2OCM 1SCM,"1 centimetre, 20 centimetre","1 centimetre, 20 centimetre",20 centimetre
2,https://m.media-amazon.com/images/I/41ADVPQgZO...,item_weight,993359,CalaLrian Chili Powder,,NaN,None
3,https://m.media-amazon.com/images/I/417NJrPEk+...,maximum_weight_recommendation,939426,Deodorizing module Cat litter shovel Adsorb ba...,,NaN,None
4,https://m.media-amazon.com/images/I/41nblnEkJ3...,voltage,648011,LED OSRAM 55w= 40w 470 Im am White B22d,,NaN,None
5,https://m.media-amazon.com/images/I/41pvwR9Gba...,voltage,965518,LED WORK LIGHT for Dark Areas 'DAKLAR |,,NaN,None
6,https://m.media-amazon.com/images/I/41nblnEkJ3...,wattage,648011,LED OSRAM 55w= 40w 470 Im am White B22d,"40 watt, 55 watt","55 watt, 40 watt",55 watt
7,https://m.media-amazon.com/images/I/41uwo4PVnu...,depth,640565,"Joz"" [",,NaN,None
8,https://m.media-amazon.com/images/I/41ygXRvf8l...,height,752266,50cm 8cm,"50 centimetre, 8 centimetre","50 centimetre, 8 centimetre",50 centimetre
9,https://m.media-amazon.com/images/I/41uwo4PVnu...,width,640565,"Joz"" [",,NaN,None


In [ ]:
df.to_csv('processed_data.csv', index=False)